In [1]:
import cv2
import os
import sys
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches

# we need to append the root directory to the path to import data.py
sys.path.append('..')
import data
import model
import utils

%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
params = utils.yaml_to_dict('../config.yml')
params['data_dir'] = os.path.join('..', params['data_dir'])
params['data_dir_lfw'] = os.path.join('..', params['data_dir_lfw'])

In [12]:
test_data_set = data._sources(params, mode='test')

In [13]:
checkpoints_dir = os.path.join('..',params['model_dir'])

In [14]:
model_names = sorted([
    name for name in os.listdir(checkpoints_dir)
    if name.startswith('model.ckpt') and name.endswith('.meta')
], key=lambda x: x.split('-')[1])

newest_model = model_names[-1][:-5]
newest_model

'model.ckpt-1500'

In [15]:
model_path = os.path.join(checkpoints_dir, newest_model)

In [16]:
estimator = tf.estimator.Estimator(
    model_fn=model.model_fn,
    model_dir=checkpoints_dir,
    params=params,
    warm_start_from=model_path
)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '..\\checkpoints', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000001487BB13CC0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [17]:
predictions = estimator.predict(
    input_fn = lambda: data.input_fn(test_data_set,False, params)
)

In [18]:
p = []
for i in predictions:
    p.append(i['embeddings'])

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ..\checkpoints\model.ckpt-1500
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [46]:
p[0].shape

(128,)

In [66]:
dist = np.linalg.norm(p[0]-p[1])

In [41]:
test_data_set

(['..\\dataset\\lfw\\Barbra_Streisand\\Barbra_Streisand_0001.jpg',
  '..\\dataset\\lfw\\Barbra_Streisand\\Barbra_Streisand_0002.jpg',
  '..\\dataset\\lfw\\Barbra_Streisand\\Barbra_Streisand_0003.jpg',
  '..\\dataset\\lfw\\Mike_Brey\\Mike_Brey_0001.jpg',
  '..\\dataset\\lfw\\Mike_Brey\\Mike_Brey_0002.jpg',
  '..\\dataset\\lfw\\Mary_Sue_Coleman\\Mary_Sue_Coleman_0001.jpg',
  '..\\dataset\\lfw\\Mary_Sue_Coleman\\Mary_Sue_Coleman_0002.jpg',
  '..\\dataset\\lfw\\Matthew_McConaughey\\Matthew_McConaughey_0001.jpg',
  '..\\dataset\\lfw\\Matthew_McConaughey\\Matthew_McConaughey_0002.jpg',
  '..\\dataset\\lfw\\Jeffrey_Ashby\\Jeffrey_Ashby_0001.jpg',
  '..\\dataset\\lfw\\Jeffrey_Ashby\\Jeffrey_Ashby_0002.jpg',
  '..\\dataset\\lfw\\McGuire_Gibson\\McGuire_Gibson_0001.jpg',
  '..\\dataset\\lfw\\McGuire_Gibson\\McGuire_Gibson_0002.jpg',
  '..\\dataset\\lfw\\Larenz_Tate\\Larenz_Tate_0001.jpg',
  '..\\dataset\\lfw\\Larenz_Tate\\Larenz_Tate_0002.jpg',
  '..\\dataset\\lfw\\Raja_Ibrahim\\Raja_Ibrahim_000